# 简介

> 指导文章：[01. 初识 LLM API：环境配置与多轮对话演示](https://github.com/Hoper-J/LLM-Guide-and-Demos/blob/master/Guide/01.%20初识%20LLM%20API：环境配置与多轮对话演示.md)

在这个 Notebook 中，我们将展示如何使用大语言模型（LLM）API，通过简单的 API 调用，演示如何与大模型进行交互。这是对 API 基础使用的一个演示，不涉及构建 AI 应用。

在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/01-llm-api) | [Colab](https://colab.research.google.com/drive/1jFC_WIec7KW08GBxHuEHikcXWbbv3gSh?usp=share_link)

# 环境变量配置

为了保护API密钥的安全，需要将其设置为环境变量。环境变量允许我们在不将敏感信息写入代码的情况下访问它们。

你可以通过两种方式之一来设置环境变量：
1. 在终端中设置环境变量。
2. 在 Python 脚本中设置环境变量。

这里直接在 Python 中进行设置。

In [ ]:
# 此方法仅在当前 Python 程序或 Notebook 中有效，其他程序或 Notebook 不会共享此设置。
# 国内大模型开启了新的篇章，考虑到以后可能会进行多模型的 API 输出对比，不再固定命名为 `OPENAI_API_KEY`。
import os

# 阿里云（通义千问） API 密钥在 OPENAI_API_KEY 设置
os.environ['OPENAI_API_KEY'] = 'your-api-key'

# 智谱 API
os.environ['ZHIPUAI_API_KEY'] = 'your-api-key'

# DEEPSEEK API
os.environ['DEEPSEEK_API_KEY'] = 'your-api-key'

# 获取环境变量

使用 `os.getenv()` 函数来获取环境变量的值，这样我们可以在代码中安全地访问API密钥。

In [ ]:
# 获取API密钥
api_key = os.getenv('OPENAI_API_KEY')
# api_key = os.getenv('ZHIPUAI_API_KEY')
# api_key = os.getenv('DEEPSEEK_API_KEY')

# 打印密钥以确认它被成功读取
print(api_key)

# 安装所需库

接下来，我们需要安装 openai 库，用于与阿里云的大模型 API 进行交互。

In [ ]:
!pip install openai
#!pip install 'httpx<0.28.0' # 降级 httpx 以解决关键字 'proxies' 被移除的问题，最新的 openai 库不会引发该问题，故默认注释

根据 API 平台执行后续对应的代码块。

# 单轮对话演示

在这一部分，我们将通过API调用构建一个简单的单轮对话，你可以输入一个问题，模型将会返回一个响应。

## 阿里 API

In [ ]:
from openai import OpenAI
import os

# 定义函数来获取模型响应
def get_response():
    client = OpenAI(
        api_key=os.getenv('OPENAI_API_KEY'),  # 获取环境变量中的API密钥
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 使用阿里云大模型API
    )
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': '你是谁？'}
        ]
    )
    print(completion.model_dump_json())

# 调用函数进行对话
get_response()

## 智谱 API

In [ ]:
from openai import OpenAI
import os

def get_response():
    client = OpenAI(
        api_key=os.getenv('ZHIPUAI_API_KEY'),
        base_url="https://open.bigmodel.cn/api/paas/v4",
    )
    completion = client.chat.completions.create(
        model="glm-4-plus",
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': '你是谁？'}]
        )
    print(completion.model_dump_json())

get_response()

## DeepSeek API


In [ ]:
from openai import OpenAI
import os

def get_response():
    client = OpenAI(
        api_key=os.getenv('DEEPSEEK_API_KEY'), # 1
        base_url="https://api.deepseek.com", # 2
    )
    completion = client.chat.completions.create(
        model="deepseek-chat", # 3
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': '你是谁？'}]
        )
    print(completion.model_dump_json())

get_response()

# 多轮对话演示

扩展上面的代码，支持多轮对话。这意味着模型可以记住上下文，从而生成更连贯的回答。

## 阿里 API

In [ ]:
def get_response(messages):
    client = OpenAI(
        api_key=os.getenv('OPENAI_API_KEY'), 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=messages
    )
    return completion

# 初始化对话历史
messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

# 进行多轮对话
for i in range(3):
    user_input = input("请输入你的问题：")
    
    # 将用户问题添加到对话历史中
    messages.append({'role': 'user', 'content': user_input})
    
    # 获取模型回复并打印
    assistant_output = get_response(messages).choices[0].message.content
    print(f'模型回复：{assistant_output}')
    
    # 将模型的回复也添加到对话历史中
    messages.append({'role': 'assistant', 'content': assistant_output})

## 智谱 API

In [ ]:
from openai import OpenAI
import os

def get_response(messages):
    client = OpenAI(
        api_key=os.getenv('ZHIPUAI_API_KEY'), 
        base_url="https://open.bigmodel.cn/api/paas/v4",
    )
    completion = client.chat.completions.create(
        model="glm-4-plus",
        messages=messages
        )
    return completion

messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]
# 您可以自定义设置对话轮数，当前为3
for i in range(3):
    user_input = input("请输入：")
    
    # 将用户问题信息添加到messages列表中，这部分等价于之前的单轮对话
    messages.append({'role': 'user', 'content': user_input})
    assistant_output = get_response(messages).choices[0].message.content
    
    # 将大模型的回复信息添加到messages列表中，这里是历史记录，保存上下文
    messages.append({'role': 'assistant', 'content': assistant_output})
    print(f'用户输入：{user_input}')
    print(f'模型输出：{assistant_output}')
    print('\n')

## DeepSeek API

In [ ]:
from openai import OpenAI
import os

def get_response(messages):
    client = OpenAI(
        api_key=os.getenv('DEEPSEEK_API_KEY'), 
        base_url="https://api.deepseek.com",
    )
    completion = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
        )
    return completion

messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}]
# 您可以自定义设置对话轮数，当前为3
for i in range(3):
    user_input = input("请输入：")
    
    # 将用户问题信息添加到messages列表中，这部分等价于之前的单轮对话
    messages.append({'role': 'user', 'content': user_input})
    assistant_output = get_response(messages).choices[0].message.content
    
    # 将大模型的回复信息添加到messages列表中，这里是历史记录，保存上下文
    messages.append({'role': 'assistant', 'content': assistant_output})
    print(f'用户输入：{user_input}')
    print(f'模型输出：{assistant_output}')
    print('\n')

# 流式输出演示

流式输出允许我们实时查看模型生成的回答，而不是等待最终结果。

## 阿里 API

In [ ]:
# 实现流式输出
def get_response_stream():
    client = OpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=[{'role': 'system', 'content': 'You are a helpful assistant.'},
                  {'role': 'user', 'content': '你是谁？'}],
        stream=True,
        stream_options={"include_usage": True}
    )
    
    # 实时输出生成的结果
    for chunk in completion:
        print(chunk.model_dump_json())

# 调用流式输出函数
get_response_stream()

## 智谱 API

In [ ]:
from openai import OpenAI
import os

def get_response():
    client = OpenAI(
        api_key=os.getenv("ZHIPUAI_API_KEY"),
        base_url="https://open.bigmodel.cn/api/paas/v4",
    )
    completion = client.chat.completions.create(
        model="glm-4-plus",
        messages=[{'role': 'system', 'content': 'You are a helpful assistant.'},
                  {'role': 'user', 'content': '你是谁？'}],
        stream=True,
        # 可选，配置以后会在流式输出的最后一行展示token使用信息
        stream_options={"include_usage": True}
        )
    for chunk in completion:
        print(chunk.model_dump_json())

if __name__ == '__main__':
    get_response()

## DeepSeek API

In [ ]:
from openai import OpenAI
import os

def get_response():
    client = OpenAI(
        api_key=os.getenv("DEEPSEEK_API_KEY"),
        base_url="https://api.deepseek.com",
    )
    completion = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{'role': 'system', 'content': 'You are a helpful assistant.'},
                  {'role': 'user', 'content': '你是谁？'}],
        stream=True,
        # 可选，配置以后会在流式输出的最后一行展示token使用信息
        stream_options={"include_usage": True}
        )
    for chunk in completion:
        print(chunk.model_dump_json())

if __name__ == '__main__':
    get_response()

# 总结与扩展学习

恭喜你完成了API的使用！在这个notebook中，我们学习了如何与大语言模型API进行交互，如何设置环境变量，如何实现单轮和多轮对话，并了解了流式输出的概念。

如果对AI和生成式人工智能感兴趣，推荐观看李宏毅老师的课程：
- [生成式人工智能导论-视频](https://www.bilibili.com/video/BV1BJ4m1e7g8)
- [课程主页](https://speech.ee.ntu.edu.tw/~hylee/genai/2024-spring.php)